In [14]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import os
from zipfile import ZipFile

import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Lambda

In [16]:
print("Unzipping file..")
with ZipFile('food.zip', 'r') as zipObj:
   #Extracting files to food folder
   zipObj.extractall()
   print('File is unzipped in food folder')


#We initialize the model for feature extraction
resnet = tf.keras.applications.InceptionResNetV2(pooling='avg',include_top=False)
resnet.trainable = False
d = input_ = Input(shape=(299,299,3))
d = resnet(d)
model = Model(inputs=input_, outputs=d)
print("Model is initialized.")

#We extract the features if it hasn't been done already
if os.path.exists('feature_list.npy'):
    feature_list = np.load('feature_list.npy')
else:   
    feature_list = []
    for i in tqdm(range(0,10000)):
        img = image.load_img(os.path.join('food',str(i).zfill(5))+'.jpg', target_size=(299,299))
        img = image.img_to_array(img)
        x = np.expand_dims(img, axis=0)
        x = tf.keras.applications.inception_resnet_v2.preprocess_input(x)
        features = resnet.predict(x)
        feature_list.append(features) 
    #Reduce dimension of feature list to 2D array   
    feature_list = np.squeeze(feature_list)   
    np.save('feature_list.npy',feature_list)

#Build train tensors
train_triplets = pd.read_csv('train_triplets.txt',header=None,sep=" ")
anchor = np.take(feature_list,train_triplets.iloc[:,0].to_numpy(),axis=0)
positive = np.take(feature_list,train_triplets.iloc[:,1].to_numpy(),axis=0)
negative = np.take(feature_list,train_triplets.iloc[:,2].to_numpy(),axis=0)
X_train1 = np.concatenate((anchor,positive,negative),axis=1)
X_train2 = np.concatenate((anchor,negative,positive),axis=1)
train_tensors=np.concatenate((X_train1,X_train2),axis=0)

#Build label tensors
y_train1 = np.ones((anchor.shape[0],1))
y_train2 = np.zeros((anchor.shape[0],1))
labels=np.concatenate((y_train1,y_train2),axis=0)


#Creating fully connected NN
d = input_ = Input(train_tensors.shape[1:])
d = Activation('relu')(d)
d = Dropout(0.7)(d)
d = Dense(1152)(d)
d = Activation('relu')(d)
d = Dense(288)(d)
d = Activation('relu')(d)
d = Dense(72)(d)
d = Activation('relu')(d)
d = Dense(18)(d)
d = Activation('relu')(d)
d = Dense(1)(d)
d = Activation('sigmoid')(d)
model = Model(inputs=input_, outputs=d)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print('Model has been successfully built.')


#Training
model.fit(x = train_tensors, y = labels, epochs=5)
print("Training completed!")

#Build test tensor
test_triplets = pd.read_csv('test_triplets.txt',header=None,sep=" ")
anchor = np.take(feature_list,test_triplets.iloc[:,0].to_numpy(),axis=0)
positive = np.take(feature_list,test_triplets.iloc[:,1].to_numpy(),axis=0)
negative = np.take(feature_list,test_triplets.iloc[:,2].to_numpy(),axis=0)
X_test = np.concatenate((anchor,positive,negative),axis=1)

#Predictions
Y_pred = model.predict(X_test)

#Saving to text file
np.savetxt('result.txt',np.where(Y_pred < 0.5, 0, 1),fmt='%d')

Unzipping file..
File is unzipped in food folder
Model is initialized.
Model has been successfully built.
Epoch 1/5
 614/3720 [===>..........................] - ETA: 1:28 - loss: 0.6835 - accuracy: 0.5547

KeyboardInterrupt: 